Проект:

Запрос сформирован в связи с тем, что просмотр фильмов на языке оригинала является популярным и эффективным методом прокачки при изучении иностранных языков. При этом важно выбрать фильм, соответствующий уровню сложности студента, т.е. чтобы студент понимал 50-70% диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и определить, какому уровню он соответствует. Однако это требует больших временных затрат.

Наша задача:

Разработать ML-решение для автоматического определения уровня сложности англоязычных фильмов.

В решении поставленной задачи могут помочь следующие шаги:
1. Загрузить данные
2. Провести EDA
3. Попытаться собрать больше данных
4. изучить способы решения аналогичных задач: работа с текстовыми данными лемматизация, стемминг, стоп-слова и т.д., библиотеки NLP.
5. Построить базовую модель, оценить качество
6. Доработать решение (повторить шаги 3-6)

1. Понимание проблемы:
Тщательно изучите проблему, включая требования, ограничения и цели. Разбейте задачу на более мелкие подзадачи и убедитесь, что у вас есть четкое понимание того, что ожидается от решения.

2. Сбор и предварительная обработка данных:
Соберите набор данных транскриптов англоязычных фильмов с соответствующими уровнями сложности. Можно использовать такие источники, как субтитры, сценарии фильмов или онлайновые базы данных. Предварительно обработайте данные, очистив текст, удалив специальные символы и преобразовав их в строчные.

3. Эксплораторный анализ данных (EDA):
Проанализируйте набор данных, чтобы получить представление о распределении уровней сложности, длине текстов и любых закономерностях, которые могут возникнуть. Этот этап поможет лучше понять данные и направить процесс разработки функций.

4. Расширение данных:
Если набор данных ограничен, рассмотрите возможность его дополнения путем создания синтетических данных или использования таких методов, как обратный перевод, для создания дополнительных примеров. Это поможет повысить эффективность модели.

5. Предварительная обработка текста:
Применяются такие методы НЛП, как токенизация, лемматизация и удаление стоп-слов. Эти действия позволяют подготовить текстовые данные для ввода в модель.

6. Извлечение признаков с помощью трансформаторов:
Использование предварительно обученных моделей трансформаторов из библиотек, таких как Hugging Face Transformers. Эти модели, такие как BERT, GPT-2 или RoBERTa, были обучены на огромных объемах текста и могут извлекать из текстовых данных богатые контекстуальные вкрапления. Настройте эти модели на конкретную задачу, чтобы повысить их производительность.

7. Архитектура модели:
Разработайте архитектуру модели. Вы можете использовать PyTorch для создания собственных архитектур нейронных сетей или модифицировать существующие архитектуры для решения конкретной задачи. Входными данными для модели будут контекстные вкрапления, сгенерированные моделью трансформатора.

8. Базовая модель:
Постройте базовую модель, используя разработанную архитектуру. Обучите модель на своем наборе данных и оцените ее производительность с помощью соответствующих метрик для задачи классификации. Это послужит отправной точкой для итеративных улучшений.

9. Оценка и доработка модели:
Проанализируйте производительность базовой модели и определите области, требующие улучшения. Экспериментируйте с различными гиперпараметрами, архитектурой модели и методиками для повышения точности модели. Для оценки устойчивости модели можно использовать такие методы, как перекрестная валидация.

10. Итерация и оптимизация:
Итеративно дорабатывайте модель, возвращаясь к шагам 3-6. Собирайте больше данных, экспериментируйте с различными методиками и настраивайте модель для достижения лучших результатов.

11. Документирование и презентация:
Документируйте каждый этап работы, включая предварительную обработку данных, архитектуру модели, гиперпараметры и результаты. Создайте четкую и лаконичную презентацию с кратким описанием вашего подхода, возникших проблем и итоговой эффективности модели.

12. Будущие усовершенствования:
Рассмотрите дополнительные возможности и методы, которые могут еще больше повысить точность модели. Можно рассмотреть ансамблевые методы, более совершенные архитектуры трансформаторов или использование внешних ресурсов.

In [3]:
import os
import shutil
import re

import pandas as pd
import numpy as np
import spacy
import pysrt
from tqdm import tqdm

import torch

import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

In [6]:
movies_labels = pd.read_csv('data/English_scores/movies_labels_big_subs.csv')

In [7]:
movies_labels

,movie,level,subs
0,10_Cloverfield_lane(2016),3,"ben on phone michelle, please don t hang up. j..."
1,10_things_I_hate_about_you(1999),1,"i ll be right with you. so, cameron. here you ..."
2,A_knights_tale(2001),1,should we help him? he s due in the lists in t...
3,A_star_is_born(2018),1,get to it. black eyes open wide it s time to t...
4,Aladdin(1992),0,where the caravan camels roam where it s flat ...
...,...,...,...
361,There.Will.Be.Blood[2007]DvDrip[Eng]-aXXo.en,3,there she is. there she is. there she is. ladi...
362,timpe-inception,3,i know what this is. i ve seen one before. man...
363,Trainspotting.1996.720p.BluRay.x264-SiNNERS.EN,3,choose a career. choose a family. choose a fuc...
364,Twelve.Monkeys.(1995).DVDRip.XviD-UnSeeN,3,"inmate number cole, james. jose, what s going ..."


In [1]:
movies_labels.duplicated().sum()

NameError: name 'movies_labels' is not defined

In [46]:
movies_labels.drop_duplicates(inplace=True)

In [47]:
level_dict = {
    'A2': 0,
    'B1': 1,
    'B2': 2,
    'C1': 3,
}

movies_labels['level'] = movies_labels['level'].map(level_dict)

In [48]:
movies_labels['level'].value_counts()

level
2    136
1     81
0     81
3     69
Name: count, dtype: int64

In [49]:
movies_labels.shape

(367, 2)

## Предобработка выборки

In [50]:
sub_names = os.listdir('data/English_scores/Subtitles_all/Subtitles')

print(f'Количество файлов: {len(sub_names)}')

Количество файлов: 364


In [51]:
sub_filtr = set(sub_names) & set(movies_labels['movie'] + '.srt')
print(f'Количество фильмов, имеющих метку и субтитры: {len(sub_filtr)}')

Количество фильмов, имеющих метку и субтитры: 363


In [52]:
oxford = load_files('data/Oxford_CEFR_level/Classic Oxford', shuffle=False, encoding='utf-8-sig')
oxford.data[0][:50]

'a, an \nabout \nabove \nacross \naction\nactivity\nactor'

In [53]:
HTML = r'<.*?>' # html тэги на пробел
TAG = r'{.*?}' # тэги на пробел
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' # комменты в скобках на пробел
LETTERS = r'[^a-zA-Z\.,!? ]' # все что не буквы на пробел
SPACES = r'([ ])\1+' # повторяющиеся пробелы на один пробел
DOTS = r'[\.]+' # многоточие на точку

In [54]:
def clean_subs(subs):
    subs = subs[1:]
    txt = re.sub(HTML, ' ', subs.text)
    txt = re.sub(COMMENTS, ' ', txt)
    txt = re.sub(LETTERS, ' ', txt)
    txt = re.sub(DOTS, r'.', txt)
    txt = re.sub(SPACES, r'\1', txt)
    txt = re.sub('www', '', txt)
    txt = txt.lstrip()
    txt = txt.encode('ascii', 'ignore').decode()
    txt = txt.lower()

    return txt

In [55]:
def lemma_count(lemmas, oxf, cat):
    func_dict = {'A1': 0,
                 'A2': 1,
                 'B1': 2,
                 'B2': 3,
                 'C1': 4}
    level = func_dict[cat]
    oxf_word_list = oxf[level].split()
    words = [lemma for lemma in lemmas if lemma in oxf_word_list]

    return len(set(words))

In [56]:
dataset_path = 'data/English_scores/Subtitles_all/Subtitles'

for film in tqdm(sub_filtr):
    try:
        subs = pysrt.open(f'{dataset_path}/{film}')
    except:
        subs = pysrt.open(f'{dataset_path}/{film}', encoding='iso-8859-1')

    cln_subs = clean_subs(subs)
    movies_labels.loc[movies_labels['movie'] == film[:-4], 'subs'] = cln_subs

    nlp = spacy.load('en_core_web_sm')
    doc = nlp(cln_subs)
    lemma_list = [token.lemma_ for token in doc]

    # for lvl in ['A2', 'B1', 'B2', 'C1']:
    #     movies_labels.loc[movies_labels['movie'] == film[:-4], lvl+'_lemma_cnt'] = lemma_count(lemma_list, oxford.data, lvl)

100%|██████████| 363/363 [11:55<00:00,  1.97s/it]


In [59]:
movies_labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 366 entries, 0 to 366
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   movie   366 non-null    object
 1   level   366 non-null    int64 
 2   subs    366 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.4+ KB


In [58]:
movies_labels.dropna(inplace=True)

In [60]:
movies_labels

,movie,level,subs
0,10_Cloverfield_lane(2016),1,"ben on phone michelle, please don t hang up. j..."
1,10_things_I_hate_about_you(1999),1,"i ll be right with you. so, cameron. here you ..."
2,A_knights_tale(2001),2,should we help him? he s due in the lists in t...
3,A_star_is_born(2018),2,get to it. black eyes open wide it s time to t...
4,Aladdin(1992),0,where the caravan camels roam where it s flat ...
...,...,...,...
362,There.Will.Be.Blood[2007]DvDrip[Eng]-aXXo.en,3,there she is. there she is. there she is. ladi...
363,timpe-inception,3,i know what this is. i ve seen one before. man...
364,Trainspotting.1996.720p.BluRay.x264-SiNNERS.EN,3,choose a career. choose a family. choose a fuc...
365,Twelve.Monkeys.(1995).DVDRip.XviD-UnSeeN,3,"inmate number cole, james. jose, what s going ..."


In [61]:
movies_labels.to_csv('data/English_scores/movies_labels_big_subs.csv', index=False)

In [4]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = text.split()
    filtered_text = [word for word in word_tokens if word not in stop_words]

    return ' '.join(filtered_text)

In [5]:
def lemmatize(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    lemma_list = [token.lemma_ for token in doc]

    return ' '.join(lemma_list)

In [6]:
movies_labels['subs'] = movies_labels['subs'].apply(remove_stopwords)

In [7]:
movies_labels['subs'] = movies_labels['subs'].apply(lemmatize)

In [8]:
movies_labels['subs_len'] = movies_labels['subs'].apply(lambda x: len(x.split()))

In [9]:
for lvl in ['A2', 'B1', 'B2', 'C1']:
    movies_labels[f'{lvl}_lemma_frequency'] = movies_labels[f'{lvl}_lemma_cnt'] / movies_labels['subs_len']

In [10]:
movies_labels

,movie,level,subs,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt,subs_len,A2_lemma_frequency,B1_lemma_frequency,B2_lemma_frequency,C1_lemma_frequency
0,10_Cloverfield_lane(2016),2,"ben phone michelle , please hang up . talk I ,...",187.0,128.0,105.0,39.0,3845,0.048635,0.033290,0.027308,0.010143
1,10_things_I_hate_about_you(1999),2,"right you . so , cameron . go . nine school ye...",241.0,176.0,160.0,68.0,6583,0.036609,0.026736,0.024305,0.010330
2,A_knights_tale(2001),3,help he ? due list two minute . two minute for...,246.0,153.0,135.0,75.0,5977,0.041158,0.025598,0.022587,0.012548
3,A_star_is_born(2018),3,get it . black eye open wide time testify room...,257.0,141.0,127.0,47.0,9883,0.026004,0.014267,0.012850,0.004756
4,Aladdin(1992),1,caravan camels roam flat immense heat intense ...,244.0,155.0,149.0,66.0,6769,0.036047,0.022899,0.022012,0.009750
...,...,...,...,...,...,...,...,...,...,...,...,...
226,Suits.S03E06.720p.HDTV.x264-mSD,4,assume deal edward dead . long win murder tria...,187.0,111.0,99.0,38.0,4493,0.041620,0.024705,0.022034,0.008458
227,Suits.S03E07.HDTV.x264-mSD,4,go wall tomorrow . way get side . want side . ...,205.0,138.0,126.0,60.0,4324,0.047410,0.031915,0.029140,0.013876
228,Suits.S03E08.480p.HDTV.x264-mSD,4,darby back manage partner . want anymore . get...,188.0,124.0,116.0,51.0,4274,0.043987,0.029013,0.027141,0.011933
229,Suits.S03E09.480p.HDTV.x264-mSD,4,"bond father here . speak take easy I , you ? l...",208.0,130.0,123.0,52.0,4566,0.045554,0.028471,0.026938,0.011389


In [8]:
movies_labels.reset_index(drop=True, inplace=True)

In [9]:
def split_text_by_n_words(text: str, n: int=10) -> list:
    tokens = text.split()
    chunked_text = []

    for i in range(0, len(tokens), n):
        chunk = " ".join(tokens[i:i+n])
        chunked_text.append(chunk)

    return chunked_text

In [10]:
texts = []
levels = []
movies = []

for i in range(movies_labels.shape[0]):
    text = movies_labels.loc[i, 'subs']
    level = movies_labels.loc[i, 'level']
    movie = movies_labels.loc[i, 'movie']

    sentences = split_text_by_n_words(text, 256)

    texts.extend(sentences)
    levels.extend([level] * len(sentences))
    movies.extend([movie] * len(sentences))

In [11]:
len(texts), len(levels), len(movies)

(11483, 11483, 11483)

In [12]:
sentences_df = pd.DataFrame({'text': texts, 'level': levels, 'movie': movies})
sentences_df.head()

,text,level,movie
0,"ben on phone michelle, please don t hang up. j...",3,10_Cloverfield_lane(2016)
1,"your wallet. given as how i saved your life, i...",3,10_Cloverfield_lane(2016)
2,"and make sure they re okay. michelle, they re ...",3,10_Cloverfield_lane(2016)
3,"he s sorry for, correct? totally. let s go. ba...",3,10_Cloverfield_lane(2016)
4,possible. i heard one earlier. above my room. ...,3,10_Cloverfield_lane(2016)


In [13]:
sentences_df['level'].value_counts()

level
2    3955
1    2722
0    2464
3    2342
Name: count, dtype: int64

In [14]:
sentences_df.to_csv('data/English_scores/sentences_big.csv', index=False)